In [ ]:
from pysradb import SRAweb
import pandas as pd
import config

In [ ]:
# load the srr_list
srr_list_fn = config.LOOP_CATALOG_DIR + 'results/fastqs/stats/read_counts/srr_id_list.v1.txt'
srr_ids = [x.strip() for x in open(srr_list_fn).readlines()][1:]

In [ ]:
# load the old data
old_fn = config.LOOP_CATALOG_DIR + 'results/fastqs/stats/read_counts/srr.read_counts.v1.tsv.bkup'
old_metadata_df = pd.read_table(old_fn)
old_metadata_df.loc[:, 'run_total_spots'] =  old_metadata_df['run_total_spots'].fillna(0).replace('', 0).astype(int)
old_srr_ids = set(old_metadata_df.run_accession.tolist())

In [65]:
# extract a list of query_srr_ids to look up
query_srr_ids = [x for x in srr_ids if x not in old_srr_ids]

In [66]:
# function to query metadata for a single SRR ID
def query_metadata(srr_id):
    return sra.search_sra(srr_id)

# Initialize SRAweb object
sra = SRAweb()

In [67]:
# Query metadata for all SRR IDs in parallel
metadata_list = []
for i, srr_id in enumerate(query_srr_ids):
    data = query_metadata(srr_id)
    metadata_list.append(data)
    print(i)

In [68]:
if len(metadata_list) > 0:
    new_metadata_df = pd.concat(metadata_list)
    new_metadata_df.drop_duplicates(subset=['run_accession'], inplace=True)
    complete_metadata_df = pd.concat([old_metadata_df, new_metadata_df])
else:
    complete_metadata_df = old_metadata_df

In [69]:
complete_metadata_df.head()

,run_accession,run_total_spots
0,SRR10008147,18392515
1,SRR10008148,19833332
2,SRR10008149,27111453
3,SRR10008150,23218511
4,SRR10008151,17159570


In [70]:
save_df = complete_metadata_df[['run_accession', 'run_total_spots']]

In [ ]:
# also saving .bkup files incase we overwrite
save_full_fn =  config.LOOP_CATALOG_DIR + '/results/fastqs/stats/read_counts/srr.read_counts.full.v1.tsv'
save_insert_fn =  config.LOOP_CATALOG_DIR + '/results/fastqs/stats/read_counts/srr.read_counts.v1.tsv'
complete_metadata_df.to_csv(save_full_fn, sep='\t', index=False)
save_df.to_csv(save_insert_fn, sep='\t', index=False)

In [72]:
save_df

,run_accession,run_total_spots
0,SRR10008147,18392515
1,SRR10008148,19833332
2,SRR10008149,27111453
3,SRR10008150,23218511
4,SRR10008151,17159570
...,...,...
1325,SRR9822270,48505194
1326,SRR9822271,44313365
1327,SRR9822272,38860622
1328,SRR9889945,138428879
